In [ ]:
! pip install -q pdfplumber

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
from langchain.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("../sample-data/pdf-books/The-Big-Book-of-MLOps-v6-082322.pdf")
pdf_contents = loader.load()

In [ ]:
embeddings = LlamaCppEmbeddings(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    )

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)

documents = text_spliter.split_documents(pdf_contents)
vectorstore = Chroma.from_documents(documents, embeddings)


In [ ]:
vectorstore.similarity_search("What is MLflow?", top_k=5)

In [ ]:
vectorstore.max_marginal_relevance_search("What is MLflow?", top_k=5)

In [ ]:
llm  = LlamaCpp(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    temperature=0,
    n_gpu_layers=2,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True
)

prompt_template = """
[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 

{context} 

Question: {question} 
Answer: [/INST]
"""

prompt = PromptTemplate.from_template(template=prompt_template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":2}),
    chain_type_kwargs={"prompt": prompt},
)



In [ ]:
from langchain.globals import set_debug
set_debug(True)

question = "What is MLflow?"
result = qa_chain({"query": question})
result["result"]